In [1]:
import requests
import time
session = requests.Session()
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [335]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 3.8 MB/s eta 0:00:00


In [5]:
### suburb and postal code will be a dictionary
HEADERS = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36",
            "accept": "application/json"}



In [120]:
def get_website(seed):
#     seed = f'https://www.domain.com.au/rent/{suburb}-vic-{postalcode}/?page={page_num}'
#     seed = f"https://www.domain.com.au/sitemap-listings-rent-2022080823.xml"
    response = session.get(seed, headers=HEADERS)
    data = response.text
    return data

### Get listing groups

In [130]:
listings_groups = []
try:
    data = get_website("https://www.domain.com.au/sitemap-listings-rent.xml")
    soup = BeautifulSoup(data, "html.parser")
    listings_groups = [x.contents[0] for x in soup.find_all('loc')]
    listings_groups = pd.DataFrame(listings_groups, columns=['links'])
    listings_groups.to_csv("../data/raw/listing_groups.csv")
except Exception as e:
    print(e)

/home/sousf/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


### Get individual listings

In [142]:
listings_groups = pd.read_csv("../data/raw/listing_groups.csv")
listings_groups['links']
listings = []
try:
    for group in listings_groups['links']:
        data = get_website(group)
        soup = BeautifulSoup(data, "html.parser")
        listings.extend([x.contents[0] for x in soup.find_all('loc') if '-vic-' in x.contents[0]])
#         print(listings)
except Exception as e:
    print(e)
    
listings

/home/sousf/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


['https://www.domain.com.au/1-sanctuary-way-ascot-vale-vic-3032-16065444',
 'https://www.domain.com.au/1206-639-little-lonsdale-street-melbourne-vic-3000-16065443',
 'https://www.domain.com.au/13-86-hotham-street-east-melbourne-vic-3002-16065438',
 'https://www.domain.com.au/60-777-whitehorse-road-mont-albert-vic-3127-16065435',
 'https://www.domain.com.au/15-vincent-avenue-st-albans-vic-3021-16065436',
 'https://www.domain.com.au/1-238-deakin-avenue-mildura-vic-3500-16065433',
 'https://www.domain.com.au/4-gila-way-truganina-vic-3029-16065421',
 'https://www.domain.com.au/unit-4-18-avoca-ave-elwood-vic-3184-16065420',
 'https://www.domain.com.au/6-percy-place-prahran-vic-3181-16047983',
 'https://www.domain.com.au/316-330-lygon-street-brunswick-east-vic-3057-15187572',
 'https://www.domain.com.au/61-dutton-lane-ascot-vale-vic-3032-16065419',
 'https://www.domain.com.au/1306-618-lonsdale-street-melbourne-vic-3000-16065416',
 'https://www.domain.com.au/38-fantail-way-brookfield-vic-3338

In [143]:
len(listings)

16025

In [145]:
listings = pd.DataFrame(listings, columns=['links'])
listings.to_csv("../data/raw/listings.csv", index=False)

### Get listing data

In [316]:
# response = session.get('https://www.domain.com.au/1-sanctuary-way-ascot-vale-vic-3032-16065444', headers=HEADERS)
response = session.get('https://www.domain.com.au/1206-639-little-lonsdale-street-melbourne-vic-3000-16065443', headers=HEADERS)

listing_data = response.json()

In [317]:
listing_data.keys()

dict_keys(['pageType', 'pageTitle', 'props', 'digitalData', 'raygunTags'])

In [318]:
listing_data['props']['id']

16065443

In [319]:
listing_data['props'].keys()

dict_keys(['user', 'loginAndReturnUrl', 'signupAndReturnUrl', 'notificationApiUrl', 'listingsMap', 'otherListingsIds', 'address', 'unitNumber', 'streetNumber', 'street', 'suburb', 'postcode', 'stateAbbreviation', 'projectColor', 'createdOn', 'modifiedOn', 'projectName', 'isStandardListing', 'isArchived', 'id', 'listingId', 'listingUrl', 'projectUrl', 'bigProjectLogo', 'smallProjectLogo', 'brandingProjectLogo', 'brandingProjectColor', 'footer', 'propertyType', 'beds', 'phone', 'agencyName', 'propertyDeveloperName', 'agencyLogo', 'agencyBanner', 'agencyProfileUrl', 'propertyDeveloperUrl', 'projectLink', 'websiteLink', 'brandingColor', 'brandingAppearance', 'limitedAgencyMode', 'listingType', 'description', 'fetchUrl', 'headline', 'tagline', 'banner', 'loanfinder', 'schoolCatchment', 'whatIsNearby', 'neighbourhoodInsights', 'locationProfileCards', 'suburbInsights', 'otherProjects', 'gallery', 'additionalLinks', 'header', 'listingSummary', 'propertyProfileUrlSlug', 'agents', 'inspection', 

### Property data
price per week, postcode/suburb, address, bedrooms, bathrooms, carpark, property type, 

In [340]:
prop_data = listing_data['props']['listingSummary']
stat = prop_data.pop("stats", None)
prop_df = pd.DataFrame(prop_data, index=[0])
# prop_df['stampDutyEstimate'] = listing_data['props']['stampDutyEstimate']
prop_df['id'] = listing_data['props']['id']
prop_df

,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,showDefaultFeatures,id
0,2,1,1,$480 per week,$480 per week,"102/279 Wellington Parade South, East Melbourn...",branded,rent,Apartment / Unit / Flat,recentlyUpdated,rent,False,0,True,14787024


In [252]:
prop_data

[{'key': 'availableFrom',
  'value': 'Sunday, 4th September 2022',
  'iso8601Value': '2022-09-04T14:00:00'},
 {'key': 'bond', 'value': 2390}]

In [240]:
prop_data

{'beds': 4,
 'baths': 3,
 'parking': 2,
 'title': '$950',
 'price': '$950',
 'address': '1 Sanctuary Way, Ascot Vale VIC 3032',
 'promoType': 'platinum',
 'listingType': 'rent',
 'propertyType': 'House',
 'status': 'new',
 'mode': 'rent',
 'isRural': False,
 'houses': 0,
 'showDefaultFeatures': True}

### Suburb Data


In [267]:
listing_data['props']['suburbInsights']

{'beds': 1,
 'propertyType': 'Unit',
 'suburb': 'Melbourne',
 'suburbProfileUrl': '/suburb-profile/melbourne-vic-3000',
 'medianPrice': 330000,
 'medianRentPrice': 340,
 'avgDaysOnMarket': 121,
 'auctionClearance': 41,
 'nrSoldThisYear': 395,
 'entryLevelPrice': 180000,
 'luxuryLevelPrice': 593000,
 'renterPercentage': 0.6970133677881032,
 'singlePercentage': 0.7628455892048701,
 'demographics': {'population': 47279,
  'avgAge': '20 to 39',
  'owners': 0.3029866322118968,
  'renters': 0.6970133677881032,
  'families': 0.2371544107951299,
  'singles': 0.7628455892048701,
  'clarification': True},
 'salesGrowthList': [{'medianSoldPrice': 429000,
   'annualGrowth': 0,
   'numberSold': 665,
   'year': '2017'},
  {'medianSoldPrice': 360000,
   'annualGrowth': -0.16083916083916083,
   'numberSold': 468,
   'year': '2018'},
  {'medianSoldPrice': 355000,
   'annualGrowth': -0.013888888888888888,
   'numberSold': 487,
   'year': '2019'},
  {'medianSoldPrice': 361000,
   'annualGrowth': 0.016901

In [277]:
i = 0
# edit this to add or remove suburb features
suburb_keys = ["suburb", "medianRentPrice"]


suburb_data = {key: listing_data['props']['suburbInsights'][key] for key in suburb_keys}
suburb_data.update(listing_data['props']['suburbInsights']["demographics"])
suburb_data.pop("clarification")
suburb_data


suburb_df = pd.DataFrame(suburb_data, index = [i])
suburb_df

,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles
0,Melbourne,340,47279,20 to 39,0.302987,0.697013,0.237154,0.762846


### Neighbourhood Data

In [281]:
listing_data['props']['neighbourhoodInsights']

{'age0To19': 0.05878031,
 'age20To39': 0.775900066,
 'age40To59': 0.138868481,
 'age60Plus': 0.0264511388,
 'longTermResident': 0.120689653,
 'owner': 0.199685544,
 'renter': 0.8003145,
 'family': 0.3400703,
 'single': 0.6599297,
 'showIncomeAndExpenses': False}

In [288]:
i = 0
neighbourhood_data = listing_data['props']['neighbourhoodInsights']
neighbourhood_data.pop('map')
neighbourhood_data.pop('showIncomeAndExpenses')
neighbourhood_df = pd.DataFrame(neighbourhood_data, index = [i])
neighbourhood_df

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single
0,0.05878,0.7759,0.138868,0.026451,0.12069,0.199686,0.800315,0.34007,0.65993


In [289]:
neighbourhood_df

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single
0,0.05878,0.7759,0.138868,0.026451,0.12069,0.199686,0.800315,0.34007,0.65993


### Nearest schools

In [292]:
closest_school = listing_data['props']['schoolCatchment']['schools']
closest_school

[{'id': '',
  'educationLevel': 'secondary',
  'name': 'Ozford College - Ozford College Campus',
  'distance': 259.55083543938684,
  'state': 'VIC',
  'postCode': '3000',
  'year': '',
  'gender': '',
  'type': 'Private',
  'address': 'Melbourne, VIC 3000',
  'url': '',
  'domainSeoUrlSlug': 'ozford-college-english-language-centre-vic-3000-52094'},
 {'id': '',
  'educationLevel': 'secondary',
  'name': 'Ozford College',
  'distance': 260.5021570858975,
  'state': 'VIC',
  'postCode': '3000',
  'year': '11-12',
  'gender': 'CoEd',
  'type': 'Private',
  'address': 'Melbourne, VIC 3000',
  'url': 'http://www.ozford.edu.au',
  'domainSeoUrlSlug': 'ozford-college-vic-3000-40714'},
 {'id': '',
  'educationLevel': 'combined',
  'name': 'Haileybury Girls College - City Campus',
  'distance': 351.66409019499406,
  'state': 'VIC',
  'postCode': '3006',
  'year': '',
  'gender': '',
  'type': 'Private',
  'address': 'West Melbourne, VIC 3006',
  'url': '',
  'domainSeoUrlSlug': 'haileybury-girls

In [331]:
closest_schools = listing_data['props']['schoolCatchment']['schools']
closest_schools

[{'id': '',
  'educationLevel': 'secondary',
  'name': 'SEDA College (Victoria) - Netball Ascot Vale',
  'distance': 407.0349574515122,
  'state': 'VIC',
  'postCode': '3032',
  'year': '',
  'gender': '',
  'type': 'Private',
  'address': 'Ascot Vale, VIC 3032',
  'url': '',
  'domainSeoUrlSlug': 'seda-college-victoria-netball-ascot-vale-vic-3032-52561'},
 {'id': '10491',
  'educationLevel': 'primary',
  'name': "St Mary's School",
  'distance': 726.0180056952819,
  'state': 'VIC',
  'postCode': '3032',
  'year': 'Prep-6',
  'gender': 'CoEd',
  'type': 'Catholic',
  'address': 'Ascot Vale, VIC 3032',
  'url': 'http://www.smascotvale.catholic.edu.au/',
  'domainSeoUrlSlug': 'st-marys-school-vic-3032-10491'},
 {'id': '940',
  'educationLevel': 'primary',
  'name': 'Ascot Vale West Primary School',
  'distance': 816.3605116347898,
  'state': 'VIC',
  'postCode': '3032',
  'year': 'Prep-6',
  'gender': 'CoEd',
  'type': 'Government',
  'address': 'Ascot Vale, VIC 3032',
  'url': 'http://w

In [310]:
i = 0
top = closest_schools[0]
top = pd.DataFrame(top, index = [i])
top

,id,educationLevel,name,distance,state,postCode,year,gender,type,address,url,domainSeoUrlSlug
0,,secondary,Ozford College - Ozford College Campus,259.550835,VIC,3000,,,Private,"Melbourne, VIC 3000",,ozford-college-english-language-centre-vic-300...


In [3]:
listings = pd.read_csv("../data/raw/listings.csv")
listings

,links
0,https://www.domain.com.au/1-sanctuary-way-asco...
1,https://www.domain.com.au/1206-639-little-lons...
2,https://www.domain.com.au/13-86-hotham-street-...
3,https://www.domain.com.au/60-777-whitehorse-ro...
4,https://www.domain.com.au/15-vincent-avenue-st...
...,...
16020,https://www.domain.com.au/4-181-great-ocean-ro...
16021,https://www.domain.com.au/53-mcewan-road-heide...
16022,https://www.domain.com.au/14-8-clyde-street-es...
16023,https://www.domain.com.au/52-121-rathdowne-st-...


In [10]:
prop_list = []
sub_list = []
neigh_list = []
school_list = []
for i in tqdm(range(0, 100)):
    try:
        response = session.get(listings['links'][i], headers=HEADERS)
        listing_data = response.json()
        if ('props' not in listing_data.keys()):
            print("No props at index: ", i, listings['links'][i])

        # create property dataset
        prop_data = listing_data['props']['listingSummary']
        prop_data.pop("stats", None)
        prop_data.pop("showDefaultFeatures", None)
        prop_data.pop("tag", None)
        prop_data['id'] = listing_data['props']['id']
        prop_list.append(prop_data)


        # create suburb dataset
        # edit this to add or remove suburb features
        suburb_keys = ["suburb", "medianRentPrice"]


        suburb_data = {key: listing_data['props']['suburbInsights'][key] for key in suburb_keys}
        suburb_data.update(listing_data['props']['suburbInsights']["demographics"])
        suburb_data.pop("clarification", None)
        suburb_data['id'] = listing_data['props']['id']
        sub_list.append(suburb_data)
    #     suburb_df = pd.DataFrame(suburb_data, index = [i])
    #     suburb_df


        # create neighbourhood dataset
        neighbourhood_data = listing_data['props']['neighbourhoodInsights']
        neighbourhood_data.pop('map', None)
        neighbourhood_data.pop('showIncomeAndExpenses', None)
        neighbourhood_data['id'] = listing_data['props']['id']
        neigh_list.append(neighbourhood_data)
    #     neighbourhood_df = pd.DataFrame(neighbourhood_data, index = [i])
    #     neighbourhood_df


        # create closest school dataset
        closest_schools = listing_data['props']['schoolCatchment']['schools']
        top = closest_schools[0]
        top['id'] = listing_data['props']['id']
        top.pop("domainSeoUrlSlug", None)
        top.pop("isRadiusResult", None)
        school_list.append(top)
        time.sleep(0.3)
    except Exception as e:
        print(e)
        
    
prop_list = pd.DataFrame(prop_list)
sub_list = pd.DataFrame(sub_list)
neigh_list = pd.DataFrame(neigh_list)
school_list = pd.DataFrame(school_list)

 40%|████████████████████████████████▊                                                 | 40/100 [00:43<00:49,  1.20it/s]

No props at index:  39 https://www.domain.com.au/15-1-hatfeild-ct-west-footscray-vic-3012-16065321
'props'


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [01:50<00:00,  1.11s/it]


In [17]:
prop_list

,beds,baths,parking,title,price,address,promoType,listingType,propertyType,status,mode,isRural,houses,id
0,4,3,2,$950,$950,"1 Sanctuary Way, Ascot Vale VIC 3032",platinum,rent,House,new,rent,False,0,16065444
1,1,1,0,$550 per week,$550 per week,"1206/639 Little Lonsdale Street, Melbourne VIC...",branded,rent,Apartment / Unit / Flat,new,rent,False,0,16065443
2,1,1,1,$475 per week,$475 per week,"13/86 Hotham Street, East Melbourne VIC 3002",platinum,rent,Apartment / Unit / Flat,new,rent,False,0,16065438
3,2,1,1,$450 per week,$450 per week,"60/777 Whitehorse Road, Mont Albert VIC 3127",branded,rent,Apartment / Unit / Flat,new,rent,False,0,16065435
4,3,1,4,$370 per week,$370 per week,"15 Vincent Avenue, St Albans VIC 3021",basic,rent,House,new,rent,False,0,16065436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2,1,1,$320 PER WEEK,$320 PER WEEK,"7/150 Rupert Street, West Footscray VIC 3012",branded,rent,Apartment / Unit / Flat,recentlyUpdated,rent,False,0,13780455
95,1,1,1,$450.00,$450.00,"817/132 Commercial Road, Prahran VIC 3181",platinum,rent,Apartment / Unit / Flat,new,rent,False,0,16065154
96,2,2,1,$450 per week,$450 per week,"7/114A Westbury Close, Balaclava VIC 3183",branded,rent,Apartment / Unit / Flat,new,rent,False,0,16065153
97,2,1,1,$530.00,$530.00,"11 Dianella Walkway, Brunswick East VIC 3057",platinum,rent,Townhouse,new,rent,False,0,16065146


In [18]:
sub_list

,suburb,medianRentPrice,population,avgAge,owners,renters,families,singles,id
0,Ascot Vale,790,14758,20 to 39,0.573788,0.426212,0.418693,0.581307,16065444
1,Melbourne,340,47279,20 to 39,0.302987,0.697013,0.237154,0.762846,16065443
2,East Melbourne,395,4963,20 to 39,0.445647,0.554353,0.361296,0.638704,16065438
3,Mont Albert,420,4847,40 to 59,0.717633,0.282367,0.526342,0.473658,16065435
4,St Albans,360,37322,20 to 39,0.639986,0.360014,0.461730,0.538270,16065436
...,...,...,...,...,...,...,...,...,...
94,West Footscray,345,11451,20 to 39,0.554197,0.445803,0.413112,0.586888,13780455
95,Prahran,300,12971,20 to 39,0.404106,0.595894,0.270018,0.729982,16065154
96,Balaclava,383,5399,20 to 39,0.447415,0.552585,0.292888,0.707112,16065153
97,Brunswick East,520,11503,20 to 39,0.436301,0.563699,0.284008,0.715992,16065146


In [19]:
neigh_list

,age0To19,age20To39,age40To59,age60Plus,longTermResident,owner,renter,family,single,id
0,0.291534,0.281150,0.313099,0.114217,0.398422,0.698876,0.301124,0.672790,0.327210,16065444
1,0.058780,0.775900,0.138868,0.026451,0.120690,0.199686,0.800315,0.340070,0.659930,16065443
2,0.156915,0.345745,0.252660,0.244681,0.477941,0.515337,0.484663,0.570946,0.429054,16065438
3,0.284697,0.261566,0.286477,0.167260,0.447316,0.619718,0.380282,0.606796,0.393204,16065435
4,0.204176,0.371230,0.229698,0.194896,0.534031,0.554622,0.445378,0.433022,0.566978,16065436
...,...,...,...,...,...,...,...,...,...,...
94,0.158140,0.553488,0.179070,0.109302,0.482993,0.479730,0.520270,0.466077,0.533923,13780455
95,0.065733,0.689655,0.209052,0.035560,0.128090,0.331754,0.668246,0.472527,0.527473,16065154
96,0.217391,0.347826,0.287625,0.147157,0.481781,0.425000,0.575000,0.463964,0.536036,16065153
97,0.149701,0.525449,0.245509,0.079341,0.414894,0.275986,0.724014,0.434137,0.565863,16065146


In [20]:
school_list

,id,educationLevel,name,distance,state,postCode,year,gender,type,address,url
0,16065444,secondary,SEDA College (Victoria) - Netball Ascot Vale,407.034957,VIC,3032,,,Private,"Ascot Vale, VIC 3032",
1,16065443,secondary,Ozford College - Ozford College Campus,259.550835,VIC,3000,,,Private,"Melbourne, VIC 3000",
2,16065438,primary,Richmond West Primary School,729.249072,VIC,3121,Prep-6,CoEd,Government,"Richmond, VIC 3121",http://www.richmondwestps.vic.edu.au
3,16065435,primary,Mont Albert Primary School,671.793634,VIC,3127,Prep-6,CoEd,Government,"Mont Albert, VIC 3127",http://www.maps.vic.edu.au
4,16065436,primary,St Albans East Primary School,206.943539,VIC,3021,Prep-6,CoEd,Government,"St Albans, VIC 3021",http://www.saeps.vic.edu.au
...,...,...,...,...,...,...,...,...,...,...,...
94,13780455,special,Hester Hornbrook Academy - Braybrook Campus,724.966477,VIC,3019,,,Private,"Braybrook, VIC 3019",
95,16065154,secondary,Prahran High School,558.475235,VIC,3181,7-9,CoEd,Government,"Windsor, VIC 3181",http://www.prahranhighschool.vic.edu.au
96,16065153,secondary,Christian Brothers' College - Balaclava Campus,100.844442,VIC,3183,,,Catholic,"Balaclava, VIC 3183",
97,16065146,primary,Our Lady Help of Christians School,75.066740,VIC,3057,Prep-6,CoEd,Catholic,"Brunswick East, VIC 3057",http://www.olbrunswickeast.catholic.edu.au


In [222]:
listing_data['props']['locationProfileCards']

{'profiles': [{'type': 'street',
   'profileUrl': 'https://www.domain.com.au/street-profile/sanctuary-walk-ascot-vale-vic-3032',
   'onMarketType': 'rent',
   'address': {'streetName': 'Sanctuary',
    'streetType': 'Wk',
    'streetTypeLong': 'Walk',
    'suburb': 'Ascot Vale',
    'postcode': '3032',
    'state': 'VIC'},
   'statistics': {'owners': 7,
    'tenants': 4,
    'propertyTypes': [{'type': 'Unit',
      'total': 2,
      'forSale': 0,
      'forRent': 0,
      'soldLast12Months': 0,
      'rentedLast12Months': 0},
     {'type': 'Townhouse',
      'total': 1,
      'forSale': 0,
      'forRent': 0,
      'soldLast12Months': 0,
      'rentedLast12Months': 0},
     {'type': 'House',
      'total': 9,
      'forSale': 0,
      'forRent': 1,
      'soldLast12Months': 1,
      'rentedLast12Months': 1},
     {'type': 'Other',
      'total': 5,
      'forSale': 0,
      'forRent': 0,
      'soldLast12Months': 0,
      'rentedLast12Months': 0}],
    'averageOwnerOccupierTenureMonths

In [313]:
# listing_data['props']['stampDutyEstimate']

In [175]:
# df = pd.DataFrame.from_dict(listing_data['props'], orient='index')
# df = df.transpose()

In [214]:
# df['suburbInsights']

0    {'beds': 4, 'propertyType': 'House', 'suburb':...
Name: suburbInsights, dtype: object